In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import RobustScaler
from AdvASLTM import *
from random import shuffle

SEQ_LEN = 20
train_ratio = 0.7
PREDICTON = 3

2023-03-15 16:07:31.875634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 16:07:39.288433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/clement/miniconda3/envs/tf/lib/
2023-03-15 16:07:39.288997: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/clement/miniconda3/envs/tf/lib/
2023-03-15 16:07:39.289013: W tensorflow/compiler/tf

In [2]:
df = pd.read_csv("BTC-Hourly.csv", parse_dates=["date"], index_col= "date")

df.sort_index(inplace = True)

# Create label
df["label"] = (df["close"] < df["close"].shift(-PREDICTON))*1

df["high"] = df["high"] / df["close"]
df["low"] = df["low"] / df["close"]
df["open"] = df["open"] / df["close"]
df["close"] = df["close"].pct_change()
df["volume"] = df["Volume BTC"] / df["Volume BTC"].rolling(48).median()
features_columns = ["open", "high", "low", "close", "volume"]
df.dropna(inplace=True)

scaler = RobustScaler().fit(df[features_columns])
df[features_columns] = scaler.transform(df[features_columns])


#Generate Sequences
sequences = []
array_features = df[features_columns].to_numpy()
array_label = df["label"].to_numpy()
for i in range(SEQ_LEN, len(df)):
    sequences.append([
        array_features[i - SEQ_LEN : i],
        array_label[i-1]
    ])
del array_features, array_label

#shuffle(sequences)
X, y = zip(*sequences)


train_limit = int(train_ratio * len(df))

X_train = np.array(X[:train_limit])
y_train = np.array(y[:train_limit])

X_validation = np.array(X[train_limit:])
y_validation = np.array(y[train_limit:])

# ALSTM

In [130]:
inputs = tf.keras.layers.Input(shape=X_train.shape[1:])
x = tf.keras.layers.Dense(32, activation = "tanh", kernel_regularizer = tf.keras.regularizers.L2(0.01))(inputs )
x = tf.keras.layers.Dropout(0.3)(x)
h = tf.keras.layers.LSTM(64, return_sequences= True, dropout= 0.4)(x)
alpha = TemporalAttentionLayer(units = 64)(h)
x = LatentLayer()([alpha,h])
x = tf.keras.layers.Dense(32, activation = "relu", kernel_regularizer = tf.keras.regularizers.L2(0.01))(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(32, activation = "relu", kernel_regularizer = tf.keras.regularizers.L2(0.01))(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(32, activation = "relu", kernel_regularizer = tf.keras.regularizers.L2(0.01))(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(2, activation = "softmax")(x)



lstm_model = tf.keras.models.Model(inputs = inputs, outputs = output)
lstm_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1E-4),
    loss= "sparse_categorical_crossentropy",
    metrics =["acc"]
)

lstm_model.fit(
    X_train, y_train,
    validation_data= (X_validation, y_validation),
    epochs= 100,
    batch_size = 128   
)

Epoch 1/100
182/182 [==============================] - 12s 33ms/step - loss: 0.6943 - acc: 0.5013 - val_loss: 0.6924 - val_acc: 0.5149
Epoch 2/100
182/182 [==============================] - 5s 27ms/step - loss: 0.6927 - acc: 0.5187 - val_loss: 0.6920 - val_acc: 0.5110
Epoch 3/100
182/182 [==============================] - 5s 28ms/step - loss: 0.6925 - acc: 0.5171 - val_loss: 0.6919 - val_acc: 0.5175
Epoch 4/100
182/182 [==============================] - 5s 27ms/step - loss: 0.6921 - acc: 0.5230 - val_loss: 0.6915 - val_acc: 0.5235
Epoch 5/100
182/182 [==============================] - 5s 28ms/step - loss: 0.6917 - acc: 0.5243 - val_loss: 0.6910 - val_acc: 0.5329
Epoch 6/100
182/182 [==============================] - 5s 27ms/step - loss: 0.6912 - acc: 0.5257 - val_loss: 0.6907 - val_acc: 0.5329
Epoch 7/100
182/182 [==============================] - 5s 28ms/step - loss: 0.6911 - acc: 0.5297 - val_loss: 0.6908 - val_acc: 0.5313
Epoch 8/100
182/182 [==============================] - 5s 27m

KeyboardInterrupt: 

# Basic LSTM

In [120]:
lstm_model = tf.keras.models.Sequential([
    # tf.keras.layers.LSTM(64, return_sequences= True, dropout= 0.4),
    # tf.keras.layers.LSTM(64, return_sequences= True, dropout= 0.4),
    tf.keras.layers.LSTM(64,dropout= 0.4, return_sequences= False),
    tf.keras.layers.Dense(2, activation = "softmax")
])


lstm_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1E-3),
    loss= "sparse_categorical_crossentropy",
    metrics =["acc"]
)


lstm_model.fit(
    X_train, y_train,
    validation_data= (X_validation, y_validation),
    epochs= 100,
    batch_size = 128,
    
)

Epoch 1/100
182/182 [==============================] - 10s 25ms/step - loss: 0.6908 - acc: 0.5305 - val_loss: 0.6907 - val_acc: 0.5341
Epoch 2/100
182/182 [==============================] - 3s 16ms/step - loss: 0.6883 - acc: 0.5432 - val_loss: 0.6902 - val_acc: 0.5376
Epoch 3/100
182/182 [==============================] - 3s 16ms/step - loss: 0.6878 - acc: 0.5456 - val_loss: 0.6921 - val_acc: 0.5333
Epoch 4/100
182/182 [==============================] - 3s 18ms/step - loss: 0.6883 - acc: 0.5406 - val_loss: 0.6901 - val_acc: 0.5315
Epoch 5/100
182/182 [==============================] - 3s 17ms/step - loss: 0.6860 - acc: 0.5481 - val_loss: 0.6972 - val_acc: 0.5324
Epoch 6/100
182/182 [==============================] - 3s 17ms/step - loss: 0.6874 - acc: 0.5440 - val_loss: 0.6912 - val_acc: 0.5388
Epoch 7/100
182/182 [==============================] - 3s 17ms/step - loss: 0.6868 - acc: 0.5448 - val_loss: 0.7008 - val_acc: 0.5317
Epoch 8/100
182/182 [==============================] - 3s 17m

KeyboardInterrupt: 

In [ ]:
index1, index2 = 100, 200
results1 = model(X_validation[index1:index1 + 1], debug = True, training = False)
results2 = model(X_validation[index2:index2 + 1], debug = True, training = False)
for key in results1.keys():
    print(f"---------------------------------------------{key}-----------------------------------------------")
    print(f"Diff mean : {tf.reduce_mean(results1[key] - results2[key])}")
    print(f"Diff std : {tf.math.reduce_std(results1[key] - results2[key])}")

---------------------------------------------x-----------------------------------------------
Diff mean : 1.6196834621950984e-05
Diff std : 0.0020502840634435415
---------------------------------------------h-----------------------------------------------
Diff mean : -1.8047420553557458e-06
Diff std : 0.0004248569021001458
---------------------------------------------alpha-----------------------------------------------
Diff mean : -1.6393514670198783e-06
Diff std : 0.0002155575784854591
---------------------------------------------e-----------------------------------------------
Diff mean : -2.4673454390722327e-06
Diff std : 0.00017854946781881154
---------------------------------------------y_pred-----------------------------------------------
Diff mean : 6.386637687683105e-05
Diff std : 0.0


In [ ]:
model.summary()

Model: "adv_alstm_model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             multiple                  80        
                                                                 
 lstm_8 (LSTM)               multiple                  2112      
                                                                 
 temporal_attention_layer_8   multiple                 288       
 (TemporalAttentionLayer)                                        
                                                                 
 latent_layer_8 (LatentLayer  multiple                 0         
 )                                                               
                                                                 
 prediction_layer_8 (Predict  multiple                 33        
 ionLayer)                                                       
                                                 